# Matchen van Slavenregisters met Paramaribo wijkregisters

Eerste pogingen ten behoeve van de HackaLOD. Gebruikte datasets:

- Suriname Slave and Emancipation Registers Dataset ([download](https://hdl.handle.net/10622/CSPBHO))
- Paramaribo Ward Registers 1828-1847 ([download](https://hdl.handle.net/10622/VLN8FD))




In [43]:
# Eerst wat handige libs importeren
import pandas as pd
from rapidfuzz import process, fuzz # sneller dan fuzzywuzzy
import time
import os
import re
from concurrent.futures import ProcessPoolExecutor, as_completed # voor multiprocessing, anders duurt 't allemaal te lang :-)

In [44]:
df_slave_registers = pd.read_csv('../data/slave_registers/Dataset.csv', delimiter=',') # voor 't gemak de filename veranderd in 'Dataset.csv' (dus zonder spaties)
df_ward_registers = pd.read_csv('../data/ward_registers/WR28-47.csv', delimiter=';')

/tmp/ipykernel_79324/4234154341.py:1: DtypeWarning: Columns (1,5,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_slave_registers = pd.read_csv('../data/slave_registers/Dataset.csv', delimiter=',') # voor 't gemak de filename veranderd in 'Dataset.csv' (dus zonder spaties)
/tmp/ipykernel_79324/4234154341.py:2: DtypeWarning: Columns (2,9,16,18,20,21,35,43,44,45,46,47,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ward_registers = pd.read_csv('../data/ward_registers/WR28-47.csv', delimiter=';')


We willen (in eerste instantie) de Wijkregisters met de corresponderende serie in de slavenregisters matchen. Vandaar voeg ik even de start- en eindjaren toe aan het dataframe voor de slavenregisters.

In [45]:
def jaar_reeks_vastleggen(serieregister):
    if '-' in serieregister:
        start, end = serieregister.split('-')
        return int(start), int(end)
    elif serieregister == '1863': # Emancipatieregister is maar één jaar
        return 1863, 1863
    return None, None

# Voeg begin- en eindjaar als aparte kolommen toe.
df_slave_registers[['StartYear', 'EndYear']] = df_slave_registers['Serieregister'].apply(
    lambda x: pd.Series(jaar_reeks_vastleggen(x.strip('"')))
)

Bij wijze van test kijken we eerst naar het jaar 1846.

Ik ga uit van de regel dat de oudste bewoner op iedere kaart in het Wijkregister  de hoofdbewoner is (hoeft natuurlijk niet waar te zijn).

In [46]:
# Maak een nieuw dataframe, alleen voor 1846 
df_ward_registers_1846 = df_ward_registers[df_ward_registers['Jaar'] == 1846].copy()
df_ward_registers_1846['FullName'] = (df_ward_registers_1846['Voornaam'].fillna('') + ' ' +
                                      df_ward_registers_1846['Tussenvoegsel'].fillna('') + ' ' +
                                      df_ward_registers_1846['Achternaam'].fillna('')).str.strip()

# 'Leeftijd' is string, maak numeriek
df_ward_registers_1846['Leeftijd'] = pd.to_numeric(df_ward_registers_1846['Leeftijd'], errors='coerce')

# Rijen waar Kaart Id of Leeftijd niet is ingevuld kunnen we niet gebruiken (check later)
df_ward_registers_1846 = df_ward_registers_1846.dropna(subset=['Kaart Id', 'Leeftijd'])

# Per Kaart Id in het Wijkregister nemen we alleen de oudste persoon (ik ga ervan uit dat dat de hoofdbewoner is, maar is natuurlijk aanname)
df_ward_registers_1846 = df_ward_registers_1846.loc[df_ward_registers_1846.groupby('Kaart Id')['Leeftijd'].idxmax()]

# Om de analyse snel te doen, sla ik de slavenregisters op in een Python dict.
slave_registers_dict = {
    idx: {
        'Name_owner': row['Name_owner'],
        'StartYear': row['StartYear'],
        'EndYear': row['EndYear'],
        'SourceId': row['Id_source'],
    } for idx, row in df_slave_registers.iterrows()
}

# En de Wijkregisters idem dito, waarbij ik ze per jaar apart opsla (nu dus alleen voor 1846)
ward_registers_by_year = {}
for ward_idx, ward_data in df_ward_registers_1846.iterrows():
    year = ward_data['Jaar']
    
    if year not in ward_registers_by_year:
        ward_registers_by_year[year] = []
    ward_registers_by_year[year].append({
        'Achternaam': ward_data['Achternaam'],
        'Voornaam': ward_data['Voornaam'],
        'FullName': ward_data['FullName'],
        'Leeftijd': ward_data['Leeftijd'],
        'WardRegister_Index': ward_data['Id'],
        'Kaart_Id': ward_data['Kaart Id']
    })


In [48]:
def process_batch(batch):
    batch_matches = [] # Om de matches in op te slaan
    for slave_idx, slave_data in batch:
        name_owner = slave_data['Name_owner']
        start_year = slave_data['StartYear']
        end_year = slave_data['EndYear']
        source_id = slave_data['SourceId']
        
        # Kijk alleen naar de relevante jaren
        for year in range(start_year, end_year + 1):
            if year in ward_registers_by_year:
                for ward_data in ward_registers_by_year[year]:
                    achternaam = ward_data['Achternaam']
                    voornaam = ward_data['Voornaam']
                    
                    # Stap 1: de achternaam moet voor 90% matchen
                    achternaam_result = process.extractOne(achternaam, [name_owner], scorer=fuzz.token_set_ratio)
                    if achternaam_result:
                        achternaam_match, achternaam_score, _ = achternaam_result
                        if achternaam_score >= 90:
                            # Stap 2: vervolgens moet de voornaam voor 80% matchen
                            voornaam_result = process.extractOne(voornaam, [name_owner], scorer=fuzz.partial_ratio)
                            if voornaam_result:
                                voornaam_match, voornaam_score, _ = voornaam_result
                                if voornaam_score >= 80:
                                    batch_matches.append({
                                        'SlaveRegister_Index': source_id,
                                        'WardRegister_FullName': ward_data['FullName'],
                                        'Owner_Name': name_owner,
                                        'Match_Type': 'Direct Match',
                                        'WardRegister_Index': ward_data['WardRegister_Index']
                                    })
                                else:
                                  # Stap 3: als dat niet lukt, check of de initialen overenkomen
                                    initials = '. '.join([name[0] for name in voornaam.split()]) + '.'
                                    
                                    # Aantal initialen tellen
                                    initials_in_name_owner = re.findall(r'\b[A-Z]\.', name_owner)
                                    
                                    # Inhoud en aantal initialen moeten matchen
                                    if initials in name_owner and len(initials_in_name_owner) == len(voornaam.split()):
                                        batch_matches.append({
                                            'SlaveRegister_Index': source_id,
                                            'WardRegister_FullName': ward_data['FullName'],
                                            'Owner_Name': name_owner,
                                            'Match_Type': 'Initials Match',
                                            'WardRegister_Index': ward_data['WardRegister_Index']
                                        })
    return batch_matches


In [49]:
batch_size = 100
slave_batches = [
    list(slave_registers_dict.items())[i:i + batch_size]
    for i in range(0, len(slave_registers_dict), batch_size)
]

all_matches = []

# Timer start
start_time = time.time()

max_processes = os.cpu_count() - 2 if os.cpu_count() > 2 else 1
print(f"Gebruikt {max_processes} processen.")

total_records = len(slave_registers_dict)
completed_batches = 0
last_report_time = start_time

with ProcessPoolExecutor(max_workers=max_processes) as executor:
    futures = {executor.submit(process_batch, batch): batch for batch in slave_batches}
    
    for future in as_completed(futures):
        result = future.result()
        if result:
            all_matches.extend(result)
        
        completed_batches += 1
        completed_records = completed_batches * batch_size
        
        # Voortgang
        current_time = time.time()
        if current_time - last_report_time >= 10:
            elapsed_time = current_time - start_time
            avg_time_per_record = elapsed_time / completed_records
            estimated_time_remaining = avg_time_per_record * (total_records - completed_records)
            print(f"Gedaan: {completed_records}/{total_records}. "
                  f"Tijd: {elapsed_time:.2f} seconden. "
                  f"Geschatte tijd te gaan: {estimated_time_remaining:.2f} seconden.")
            last_report_time = current_time


matches_df = pd.DataFrame(all_matches)
total_time = time.time() - start_time
print(f"Afgerond in {total_time:.2f} seconden.")


Gebruikt 20 processen.
Afgerond in 7.85 seconden.


In [50]:
matches_df

,SlaveRegister_Index,WardRegister_FullName,Owner_Name,Match_Type,WardRegister_Index
0,071029a14417,Carel Jacobus Leijsner,Leijsner Carel Jacobus,Direct Match,93236
1,071029a14418,Carel Jacobus Leijsner,Leijsner Carel Jacobus,Direct Match,93236
2,060117a11092,Constantie van Bakker,Bakker Constantie van,Direct Match,95212
3,060117a11093,Constantie van Bakker,Bakker Constantie van,Direct Match,95212
4,060270a12458,Catharina Ulrico Buckland,Buckland Catharina Ulrica van,Direct Match,92826
...,...,...,...,...,...
3636,414927a144969,Alexander Salomons,Salomons Perla Machiel Alexander,Direct Match,90750
3637,414927a144969,Isak Mahiel Alexander Salomons,Salomons Perla Machiel Alexander,Direct Match,91195
3638,414946a162273,M. E. Pinto,Pinto M. E.,Direct Match,93646
3639,414946a162274,M. E. Pinto,Pinto M. E.,Direct Match,93646


In [51]:
matches_df.to_excel("matched.xlsx")